In [186]:
import fitz
import re

In [187]:
doc = fitz.open(r'C:\Users\marcu\Python_Scripts\H332 X-PRESS MULHACEN 11.1.2022.pdf')

In [188]:
shipper = (33, 140)
unit = (222, 290)
goods = (312, 445)
netw = (468, 502)
tare = (510, 547)
pod = (460, 550)
base_triangle = (33, 100, 547, 580) # första rektangeln på första bladet, mellan "Shipper :" och "Printed :". Lägger sedan till på y0 och y1

In [234]:
total_starts = []   # lista över rektanglar för "Shipper :" på alla blad
total_stops = []    # lista över rektanglar för "Units :" på alla blad
total_height = 0.0  # loop över summan av höjderna på alla blad
total_words = []    # lista över alla dokumentets ord på alla blad
total_pods = []     # lista över rektanglar för "Port of Discharge" på alla blad
words_in_rectangles = [] # lista för ord inom rektanglarna, utan "headers"

for page in doc:

    # starts
    starts = page.search_for("Shipper :")
    for rect in starts:
        total_starts.append(rect + (0, total_height, 0, total_height)) # lägger till på höjden för y0 och y1

    # stops
    stops = page.searchFor("Units :")
    for rect in stops:
        total_stops.append(rect + (0, total_height, 0, total_height))

    # total_words
    words_in_page = page.get_text("words")
    for words in words_in_page:
        total_words.append([words[0], words[1] + total_height, words[2], words[3] + total_height, words[4]])

    # total_pods
    pods = page.searchFor('Port of discharge')
    for rect in pods:      
        total_pods.append(rect + (0, total_height + 12.20998, 0, total_height + 16.08598)) # lägger till på y0 och y1 för att hitta POD
    
    # words_in_rectangles
    base_triangle = (33, 100 + total_height, 547, 580 + total_height) # Utgår från basrektangel (33, 100, 547, 580) och lägger på totalhöjden
    words_in_rectangles += [word for word in total_words if fitz.Rect(word[:4]).intersect(base_triangle)] # Tar fram alla ord inom rektangeln ovan

    total_height += page.rect.height # lägger till totalhöjden varje loop


#total_pods 2
lista_pods = [' '.join([words[4] for words in total_words if fitz.Rect(words[:4]).intersect(pod)]) for pod in total_pods] # Tar fram alla POD is en lista
zipped_list_pods = list(zip(total_pods, lista_pods)) # Lägger ihop rektanglarna total_pods med lista_pods




In [183]:
def gather_data(start, stop):
    rektangel = (464, start[1], 555, stop[3]) # rektangel för området
    samma_rad = start[1]        # värdet uppdateras varje loop med värdet från föregående loop
    lista_gathered_data = []    # varje ny rad representerar en instans i listan
    ordlista = ""
    
    for word in total_words:
            
        if fitz.Rect(word[:4]).intersects(rektangel):
            
            if samma_rad != word[1]:
                lista_gathered_data.append(ordlista.strip())
                ordlista = ""
                ordlista = word[4]

            elif samma_rad == word[1]:
                ordlista += " " + word[4]

            samma_rad = word[1]

    lista_gathered_data.append(ordlista.strip()) # lägger till den sista raden i loopen
       
    return lista_gathered_data

In [241]:
print(total_stops[0])

Rect(223.55999755859375, 404.6467590332031, 249.42071533203125, 416.064697265625)


In [ ]:
if word[4] == "Customs":
                customs = word[3]
                exit

In [326]:
netw_rect = (468, 100, 502, 416)



def get_goods_info():

    goods_rect = (312, 102, 445, 416)
    samma_rad = goods_rect[1]
    lista_gathered_data = []
    ordlista = ""
    more_list = []

    for word in words_in_rectangles:

        if fitz.Rect(word[:4]).intersect(goods_rect):
            
            if samma_rad != word[1]:
                    lista_gathered_data.append(ordlista.strip())
                    ordlista = ""
                    ordlista = word[4]

            elif samma_rad == word[1]:
                ordlista += " " + word[4]

            samma_rad = word[1]

    lista_gathered_data.append(ordlista.strip()) # lägger till den sista raden i loopen
    lista_gathered_data.remove('') # tar bort första "" i listan
    more_list = lista_gathered_data

    # leta om Customs status finns i lista och ta bort .pop[:20] för att ta bort från lista, men dubblera listan först.

    return lista_gathered_data

get_goods_info()

#print(words_in_rectangles)

<re.Match object; span=(0, 14), match='Customs status'>


["20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 "20' DC",
 '2 PKG STAINLESS STEEL',
 'Customs status "N"',
 'Transhipment by MSC PETRA',
 'Voy. NT202A',
 'ETD 25.1.2022 GEBZE ETA',
 '1.2.2022',
 'Ref. 11221059091 OPS',
 '43914/1631983',
 'FREIGHT PAYABLE AT As per C/P',
 'MRN: 22FI000000020567E6']